# Cleaning up doelpunten.csv file

In [44]:
import pandas as pd

### Creating a dataframe

In [45]:
df = pd.read_csv("doelpunten.csv", encoding='latin1', 
names=['Match_ID', 'Seizoen', 'Speeldag', 'Datum', 'Startuur', 'Thuisploeg', 'Uitploeg', 
'Scorende ploeg', 'Tijdstip goal', 'Score thuisploeg', 'Score Uitploeg'])

In [46]:
df.head()

,Match_ID,Seizoen,Speeldag,Datum,Startuur,Thuisploeg,Uitploeg,Scorende ploeg,Tijdstip goal,Score thuisploeg,Score Uitploeg
0,4008203,60/61,1,4/09/1960,15:00,St-Truidense VV,Standard Luik,Standard Luik,15:14,0,1
1,4008203,60/61,1,4/09/1960,15:00,St-Truidense VV,Standard Luik,Standard Luik,15:19,0,2
2,4008204,60/61,1,4/09/1960,15:00,Eendracht Aalst,Antwerp FC,Antwerp FC,15:18,0,1
3,4008204,60/61,1,4/09/1960,15:00,Eendracht Aalst,Antwerp FC,Eendracht Aalst,15:18,1,1
4,4008204,60/61,1,4/09/1960,15:00,Eendracht Aalst,Antwerp FC,Eendracht Aalst,15:26,2,1


In [47]:
df.dtypes

Match_ID             int64
Seizoen             object
Speeldag             int64
Datum               object
Startuur            object
Thuisploeg          object
Uitploeg            object
Scorende ploeg      object
Tijdstip goal       object
Score thuisploeg     int64
Score Uitploeg       int64
dtype: object

In [48]:
# tijdstip doelpunt in geheel getal
def convert(row):
    startuur = row[4]
    uur, minuut = map(int, row[8].split(':'))
    start_uur, start_minuut = map(int, startuur.split(':'))

    if uur < start_uur:
        absolute_minuten = (60 - start_minuut) + (uur + 1 - start_uur) * 60 + minuut
    else:
        absolute_minuten = (uur - start_uur) * 60 + minuut - start_minuut
    
    return absolute_minuten

In [49]:
df['Minuut goal'] = df.apply(convert, axis=1)

In [50]:
controle_binnen_tijd = df[df['Minuut goal'] > 120]
print(len(controle_binnen_tijd))

0


In [51]:
wedstrijden_df = pd.read_csv('wedstrijden.csv', encoding='latin1',
names=['Match_ID', 'Seizoen', 'Speeldag', 'Datum', 'Startuur', 'Thuisploeg', 
'Score thuisploeg', 'Score Uitploeg', 'Uitploeg'])

In [52]:
df_unique = df.drop_duplicates(subset=['Match_ID'])
wedstrijden_df_unique = wedstrijden_df.drop_duplicates(subset=['Match_ID'])

# Convert Match_ID column in wedstrijden_df_unique to int64 data type
wedstrijden_df_unique['Match_ID'] = pd.to_numeric(wedstrijden_df_unique['Match_ID'], errors='coerce')

wedstrijden_df_unique = wedstrijden_df_unique.dropna(subset=['Match_ID'])

wedstrijden_df_unique['Match_ID'] = wedstrijden_df_unique['Match_ID'].astype('int64')

# Merge DataFrames
merged_df = pd.merge(df_unique, wedstrijden_df_unique, on='Match_ID', how='inner')

C:\Users\simon\AppData\Local\Temp\ipykernel_8360\4134516261.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wedstrijden_df_unique['Match_ID'] = pd.to_numeric(wedstrijden_df_unique['Match_ID'], errors='coerce')


In [53]:
# Datums van beide dataframes vergelijken
merged_df['Datums_gelijk'] = merged_df['Datum_x'] == merged_df['Datum_y']

print(merged_df[merged_df['Datums_gelijk'] == False])

Empty DataFrame
Columns: [Match_ID, Seizoen_x, Speeldag_x, Datum_x, Startuur_x, Thuisploeg_x, Uitploeg_x, Scorende ploeg, Tijdstip goal, Score thuisploeg_x, Score Uitploeg_x, Minuut goal, Seizoen_y, Speeldag_y, Datum_y, Startuur_y, Thuisploeg_y, Score thuisploeg_y, Score Uitploeg_y, Uitploeg_y, Datums_gelijk]
Index: []

[0 rows x 21 columns]


In [69]:
# Merge DataFrames
merged_df = pd.merge(wedstrijden_df_unique, df, on='Match_ID', how='inner')

max_scores_df = merged_df.groupby('Match_ID').agg({'Score thuisploeg_x': 'max', 'Score thuisploeg_y': 'max', 
                                                   'Score Uitploeg_x': 'max', 'Score Uitploeg_y': 'max'})

# Controleren of uiteindelijke resultaat overeenkomt met doelpunten
fouten = max_scores_df[(max_scores_df['Score thuisploeg_x'] != max_scores_df['Score thuisploeg_y']) | 
                        (max_scores_df['Score Uitploeg_x'] != max_scores_df['Score Uitploeg_y'])]

if len(fouten) == 0:
    print("Geen fouten")
else:
    print(fouten.index)

Index([4008204, 4008209, 4008210, 4008211, 4008212, 4008213, 4008214, 4008444,
       4008445, 4008446,
       ...
       4011110, 4011115, 4011120, 4011126, 4011128, 4011129, 4011131, 4011142,
       4011158, 4011165],
      dtype='int64', name='Match_ID', length=315)
